# Simple Distributed Communication

This notebook implements a basic distributed system where agents can communicate over a simulated network. We'll use Python's asyncio library to handle concurrent operations and simulate network behavior.

## References:
1. Van Steen, M., & Tanenbaum, A. S. (2017). Distributed systems. Maarten van Steen Leiden, The Netherlands.
2. Attiya, H., & Welch, J. (2004). Distributed computing: fundamentals, simulations, and advanced topics (Vol. 19). John Wiley & Sons.
3. Python asyncio documentation: https://docs.python.org/3/library/asyncio.html

In [1]:
import asyncio
import random
import time
from typing import List, Dict

print("Simple Distributed Communication Simulation")

Simple Distributed Communication Simulation


## Agent Node

We'll start by defining an `AgentNode` class that represents an individual agent in our distributed system. Each agent will have a unique ID and the ability to send and receive messages.

In [2]:
class AgentNode:
    def __init__(self, id: int):
        self.id = id
        self.peers: List[int] = []  # List of peer IDs
        self.received_messages: List[str] = []
    
    async def send_message(self, to_id: int, message: str):
        # In a real system, this would use actual network communication
        # For our simulation, it will be handled by the NetworkSimulator
        print(f"Agent {self.id} sending message to Agent {to_id}: {message}")
        await asyncio.sleep(0.1)  # Simulate network delay
    
    async def receive_message(self, from_id: int, message: str):
        print(f"Agent {self.id} received message from Agent {from_id}: {message}")
        self.received_messages.append(f"From {from_id}: {message}")
    
    async def run(self):
        while True:
            if self.peers:
                to_id = random.choice(self.peers)
                message = f"Hello from {self.id} at {time.time()}"
                await self.send_message(to_id, message)
            await asyncio.sleep(random.uniform(1, 3))  # Random delay between messages

## Network Simulator

Now, let's create a `NetworkSimulator` class that will manage our agents and simulate the network communication between them.

In [3]:
class NetworkSimulator:
    def __init__(self):
        self.nodes: Dict[int, AgentNode] = {}
    
    def add_node(self, node: AgentNode):
        self.nodes[node.id] = node
    
    def setup_peers(self):
        for node in self.nodes.values():
            node.peers = [id for id in self.nodes.keys() if id != node.id]
    
    async def simulate_network(self):
        # Simulate message passing between nodes
        while True:
            for node in self.nodes.values():
                if node.peers:
                    to_id = random.choice(node.peers)
                    message = f"Network message from {node.id} at {time.time()}"
                    await self.nodes[to_id].receive_message(node.id, message)
            await asyncio.sleep(0.5)  # Simulate network interval
    
    async def run_simulation(self, duration: int):
        # Start all agent tasks
        agent_tasks = [asyncio.create_task(node.run()) for node in self.nodes.values()]
        
        # Start network simulation task
        network_task = asyncio.create_task(self.simulate_network())
        
        # Run for specified duration
        await asyncio.sleep(duration)
        
        # Cancel all tasks
        for task in agent_tasks:
            task.cancel()
        network_task.cancel()
        
        # Wait for tasks to be cancelled
        await asyncio.gather(*agent_tasks, network_task, return_exceptions=True)
        
        print("Simulation completed.")

## Running the Simulation

Now let's set up our network and run a simulation.

In [4]:
async def main():
    simulator = NetworkSimulator()
    
    # Create and add nodes
    for i in range(5):
        simulator.add_node(AgentNode(i))
    
    # Setup peer relationships
    simulator.setup_peers()
    
    # Run simulation for 10 seconds
    await simulator.run_simulation(10)
    
    # Print received messages for each node
    for node in simulator.nodes.values():
        print(f"\nAgent {node.id} received messages:")
        for msg in node.received_messages:
            print(f"  {msg}")

# Run the simulation
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

## Analysis and Observations

Let's analyze some aspects of our distributed communication system:

In [ ]:
def analyze_communication(simulator: NetworkSimulator):
    total_messages = sum(len(node.received_messages) for node in simulator.nodes.values())
    avg_messages = total_messages / len(simulator.nodes)
    
    print(f"Total messages exchanged: {total_messages}")
    print(f"Average messages per node: {avg_messages:.2f}")
    
    # Find the node with the most received messages
    max_node = max(simulator.nodes.values(), key=lambda n: len(n.received_messages))
    print(f"Node {max_node.id} received the most messages: {len(max_node.received_messages)}")

# Run analysis
analyze_communication(simulator)

## Conclusion

In this notebook, we implemented a simple distributed communication system using Python's asyncio library. Some key points:

1. We used asynchronous programming to simulate concurrent operation of multiple agents.
2. The `NetworkSimulator` class acts as a centralized coordinator, which in a real distributed system would be replaced by actual network protocols.
3. We simulated network delays and random intervals between messages to mimic real-world conditions.
4. The system demonstrates basic peer-to-peer communication in a distributed environment.

Areas for potential improvement and expansion:
1. Implement more sophisticated network topologies (e.g., ring, mesh, or tree structures).
2. Add message routing algorithms for more complex network structures.
3. Introduce network partitions and reconciliation mechanisms.
4. Implement distributed algorithms like leader election or consensus protocols.
5. Add security features like message encryption and authentication.

This implementation serves as a foundation for exploring distributed systems concepts and can be integrated with other components, such as the compositional learning system we developed earlier.